In [8]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()

In [9]:
import os 
print(os.getcwd())

c:\Users\Auguste Verdier\Desktop\TeleoClassification\scripts\MMseqs2


In [10]:
import pandas as pd
import numpy as np

In [11]:
folds = [f'fold_{i}' for i in range(1, 7)]

In [5]:


def get_ncbi_id_from_taxon_name(taxon_name):

    return ncbi.get_name_translator([taxon_name])[taxon_name][0]

def get_lineage(taxon_id):
    return ncbi.get_lineage(taxon_id)

def get_rank(taxon_id):
    return ncbi.get_rank([taxon_id])

def get_taxon_name(taxon_id):
    return ncbi.get_taxid_translator([taxon_id])[taxon_id]

def get_taxon_name_from_lineage(lineage):
    return [get_taxon_name(taxon_id) for taxon_id in lineage]

def get_rank_from_lineage(lineage):
    return {taxon_id: rank for taxon_id, rank in get_rank(lineage).items()}




In [12]:
def csv_to_fasta(csv, fasta):
    with open(fasta, 'w') as f:
        for unique_family in csv['family'].unique():
            id = 1
            family_id = get_ncbi_id_from_taxon_name(unique_family)
            family_subset = csv[csv['family'] == unique_family]
            for index, row in family_subset.iterrows():
                csv.loc[index, 'unique_header'] = f'{unique_family}.{id}'
                unique_header = f'{unique_family}.{id}'
                f.write(f'>{unique_header}\n{row["sequence"]}\n')
                id += 1
    return csv

def test_csv_to_fasta(csv,fasta):
    with open(fasta, 'w') as f:
        
        for index, row in csv.iterrows():
            
            f.write(f'>{index}\n{row["sequence"]}\n')
    return csv

for fold in folds:
    train = pd.read_csv(f'teleo_clean/{fold}/train.csv')
    train['tax_id'] = train['family'].apply(get_ncbi_id_from_taxon_name)
    csv = csv_to_fasta(train, f'teleo_clean/{fold}/train.fasta')
    csv_uniq_tax = csv[['unique_header', 'tax_id']]
    csv_uniq_tax.to_csv(f'teleo_clean/{fold}/train_tax.tsv',sep=' ', index=False)
    test = pd.read_csv(f'teleo_clean/{fold}/test.csv')
    test_csv_to_fasta(test, f'teleo_clean/{fold}/test.fasta')


# csv = csv_to_fasta(train_1, f'6_fold_teleo/{fold}/train.fasta')
# csv_uniq_tax = csv[['unique_header', 'tax_id']]
# csv_uniq_tax.to_csv(f'6_fold_teleo/{fold}/train_tax.tsv',sep=' ', index=False)


In [8]:
csv_uniq_tax= pd.read_csv(f'teleo_clean/{fold}/train_tax.tsv',sep=' ')
csv_uniq_tax.sample(5)

,unique_header,tax_id
3503,Labridae.86,8247
4882,Cichlidae.114,8113
1339,Loricariidae.145,31002
6933,Callionymidae.19,30908
1941,Gobionidae.37,2743714
